In [1]:
import tensorflow as tf
from datetime import timedelta
import numpy as np
tf.__version__

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


'2.5.0-rc1'

## Preparing X, Y

In [18]:
print("Enter timestep : ",end=' ')
timesteps = int(input())
print(timesteps)

Enter timestep :  2


In [19]:
import pandas as pd
from collections import defaultdict

provinces = ['Bangkok','Chanthaburi','Chiang Mai','Kanchanaburi','Songkhla']
path = './data/Train/fire_integrated/'
data = {}

X = defaultdict(lambda: list())
Y = defaultdict(lambda: list())

hour_step = timedelta(hours=timesteps-1)

predict_at = pd.date_range(start="2016-03-04 00:00:00", end="2019-03-17 23:00:00", freq='6H')

for province in provinces:
    df = pd.read_csv(path+f'{province}_fire_integrated.csv', index_col=0, parse_dates=True)
    data[province] = df

    for base in predict_at:
        x = df.loc[base-hour_step: base].drop(['PM2.5'], axis=1)
        y = df.loc[base+timedelta(hours=1): base+timedelta(hours=72), 'PM2.5']

        X[province].append(x)
        Y[province].append(y)


In [20]:
X['Bangkok'][2]

,Temp(C),WindDir,Wind Speed(km/h),Cambodia_frp,Myanmar_frp,Thailand_frp,Lao_PDR_frp
2016-03-04 11:00:00,30.7,105.0,13.0,8.573155,7.60015,6.686076,13.388824
2016-03-04 12:00:00,30.7,105.0,13.0,8.573155,7.60015,6.686076,13.388824


In [21]:
Y['Bangkok'][2]

2016-03-04 13:00:00    24.4
2016-03-04 14:00:00    32.8
2016-03-04 15:00:00    32.9
2016-03-04 16:00:00    30.7
2016-03-04 17:00:00    29.2
                       ... 
2016-03-07 08:00:00    36.8
2016-03-07 09:00:00    30.9
2016-03-07 10:00:00    29.0
2016-03-07 11:00:00    28.6
2016-03-07 12:00:00    27.6
Name: PM2.5, Length: 72, dtype: float64

## Reshaping input

In [177]:
bkk_x = X['Bangkok']
bkk_y = Y['Bangkok']

n_timesteps = 3
n_features = bkk.shape[1]

timesteped = []

for start in range(0, len(bkk_x.index), n_timesteps):
    timesteped.append(list(bkk_x.iloc[start: start+n_timesteps].values))

In [178]:
print(f'{timesteped[0]} predicts \n\n{bkk_y[n_timesteps-1]}')

[array([27.3       , 70.        , 15.        ,  1.57423398,  1.23852814,
        4.12      ,  1.27484536]), array([ 25.8       , 115.        ,  11.        ,   9.12968326,
         1.23852814,   8.49336735,   8.1157485 ]), array([ 30.7       , 105.        ,  13.        ,   8.57315534,
         7.60014966,   6.68607649,  13.38882353])] predicts 

2016-03-04 13:00:00    24.4
2016-03-04 14:00:00    32.8
2016-03-04 15:00:00    32.9
2016-03-04 16:00:00    30.7
2016-03-04 17:00:00    29.2
                       ... 
2016-03-07 08:00:00    36.8
2016-03-07 09:00:00    30.9
2016-03-07 10:00:00    29.0
2016-03-07 11:00:00    28.6
2016-03-07 12:00:00    27.6
Name: PM2.5, Length: 72, dtype: float64


In [21]:
from sklearn.preprocessing import MinMaxScaler

pm = df['PM2.5']

scaler = MinMaxScaler().fit(pm)

train_scaled = scaler.transform(pm)

In [22]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# define generator
n_input = 2921
n_features = 1
generator = TimeseriesGenerator(train_scaled, train_scaled, length=n_input, batch_size=100)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

# define model
model = Sequential([
    LSTM(100, activation='relu', input_shape=(n_input, n_features)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(generator,epochs=5, batch_size=100)

Test

In [31]:
df.head(4)

,Temp,WindDir,Wind Speed,PM2.5
2016-03-03 07:00:00,26.4,65,15,62.900000
2016-03-03 10:00:00,31.4,75,13,52.966667
2016-03-03 13:00:00,34.1,70,12,35.266667
2016-03-03 16:00:00,34.6,60,12,33.050000


In [25]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [28]:

# define generator
n_input = 2921
n_features = 1
generator = TimeseriesGenerator(x, y, length=2, batch_size=1)

In [33]:
X, Y = generator[1]

In [34]:
X

array([[[31.4, 75. , 13. ],
        [34.1, 70. , 12. ]]])

In [35]:
Y

array([33.05])